## 6. Using Time Travel

### 델타 레이크 리텐션 관련 안전장치
1. 델타 레이크 엔진의 경우, 로그 및 데이터의 최근 7일 이내의 로그 및 데이터는 삭제는 할 수 없다
1. 만일, 7일 이내의 기간에 대한 로그 및 데이터 삭제를 위해서는 'spark.databricks.delta.retentionDurationCheck.enabled' 설정을 false 로 해야만 한다
1. 그 이상의 기간에 대해서는 설정으로 변경 시에는 언제든지 변경이 가능하다
1. 생성 및 수정을 통한 `TBLPROPERTIES ('delta.logRetentionDuration'='10 minutes')` 변경은 초 단위까지 가능하지만
1. 명령어 `VACUUM` 실행 시에는 `VACUUM tableName RETAIN 0 HOURS;` 와 같이 시간 단위로만 변경이 가능하다

### 데이터 리텐션의 특징
1. 편의상 리텐션 테스트를 위해 `retentionDurationCheck.enabled` 값을 false 설정으로 했다고 가정 합니다
1. 데이터 파일의 경우 `TBLPROPERTIES('delta.deletedFileRetentionDuration' = '30 seconds')` 와 같이 설정 후, `VACUUM` 수행 시에 즉각적으로 삭제가 된다

### 로그 리텐션의 특징
1. 아카이브 로그의 기본 설정 상, 10회에 1번씩 체크포인팅이 되기 때문에 그 와중에 발생한 커밋에 대한 시간 여행이 불가능합니다
1. 또한 SQL 수준에서 명시적으로 `checkpont` 실행은 어렵기 때문에 API 통해서 별도 프로세스를 통해서 수행하거나, 자동 실행을 기대해야 합니다
1. 로그의 경우 해당 로그가 생성되는 시점에 `logRetentionDuration` 값에 따라서 로그가 삭제되며, 해당 정보는 `TBLPROPERTIES` 정보를 활용합니다
1. 기본 로그 삭제 설정이 30일이므로 생성시에 수정하거나, 명시적으로 변경해 주어야 운영관리에 문제가 발생하지 않습니다
   - 특히 스트리밍 애플리케이션의 경우 로그 파일이 상당히 많아질 수 있고, 하이브 테이블로 생성된 경우 조회 성능에 영향을 줄 수 있습니다
1. 2024년 10월 30일 현재 구현상 [MetadataCleanup.scala](https://github.com/delta-io/delta/blob/master/connectors/standalone/src/main/scala/io/delta/standalone/internal/MetadataCleanup.scala#L50) 구현상, 즉각적인 로그 삭제는 불가능합니다
   - 구현 내역에 따르면 현재시간을 기준으로 리텐션 시간을 뺀 시간에서 GMT 기준 0시 기준으로 일자로 이전일까지만 삭제 대상으로 지정합니다
   - 현재 체크포인트가 생성되는 시점에 로그 리텐션이 0시간 이라고 하더라도, 오늘 새벽 0시 이전의 로그만 삭제된다고 보면 됩니다
1. 정리하면 아카이브 로그의 삭제는 아래의 제약 조건이 만족해야 로그 삭제 대상으로 선정된다
   - 기본 로그 리텐션은 30일이므로 테이블 생성 시에 명시적으로 지정해 주어야 한다
   - 아카이브 로그 삭제는 개별 커밋로그 생성 시점이 아니라, 체크포인트 생성 시점이므로 트랜잭션이 없다면 지연이 발생할 수 있으며 필요하다면 명시적인 체크포인팅 수행이 필요합니다
   - 모든 조건이 만족하더라도 현재 구현상 한국시간 기준 오전 9시(GMT 기준 0시) 정각에 체크포인팅이 발생하는 경우 리텐션 시간 이전 로그가 삭제됩니다

### 스트리밍 애플리케이션의 적절한 로그와 데이터 관리 정책
1. 스트리밍 애플리케이션의 경우 로그 생성량에 따라 조정해야 하지만, 1분 이내 여러 트랜잭션이 발생하는 경우 로그 리텐션 조정 검토가 필요함
1. 특히 델타 레이크 커넥터를 통한 하이브 서비스를 고려한다면, 로그의 수에 따른 성능 저하가 예상되므로 생성 시점에 1일 이하로 리텐션을 검토할 것
1. 데이터 리텐션의 경우는 로그 리텐션 보다 길 수 없으므로 동일하게 조정하는 것이 적절합니다
1. 외에도 스트리밍 처리는 자동 압축이나, 최적화 및 컴팩션 등의 배치 처리를 별도 스케줄로 운영해야 합니다

### 로그/데이터 리텐션 강제 실행이 가능한가?
* 데이터는 가능하지만, 로그는 어렵다


In [1]:
import findspark
findspark.init()

import os
print(os.environ['JAVA_HOME'])
print(os.environ['SPARK_HOME'])

/usr/lib/jvm/java-11-openjdk-amd64
/usr/local/spark


In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

from delta import *

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]
warehouse_dir = f"{work_dir}/spark-warehouse"

# Create spark session with hive enabled
builder = (
    SparkSession
    .builder
    .appName("pyspark-notebook")
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.databricks.delta.retentionDurationCheck.enabled", "true")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.warehouse.dir", warehouse_dir)
    .enableHiveSupport()
)

In [3]:
# 델타 레이크 생성시에 반드시 `configure_spark_with_delta_pip` 구성을 통해 실행되어야 정상적인 델타 의존성이 로딩됩니다
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [4]:
# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark.conf.set("spark.sql.decimalOperations.allowPrecisionLoss", "true")
spark

In [46]:
def show(queries, num_rows = 20):
    for query in queries.split(";"):
        spark.sql(query).show(num_rows, truncate=False)

def sql(query):
    return spark.sql(query)

def history(dbName, tableName):
    return spark.sql("describe history {}.{}".format(dbName, tableName))

def table(dbName, tableName):
    return spark.read.format("delta").table("{}.{}".format(dbName, tableName))

def describe(dbName, tableName, extended = True, num_rows = 20):
    if extended:
        show("describe extended {}.{}".format(dbName, tableName), num_rows)
    else:
        show("describe {}.{}".format(dbName, tableName), num_rows)

def ls(target):
    !ls -al {target}

def ls_and_head(target, lineno):
    !ls -al {target} | grep -v 'crc' | head -{lineno}

def cat(filename):
    !cat {filename}

def grep(keyword, filename):
    !grep -i {keyword} {filename}

def grep_and_json(keyword, filename):
    !grep {keyword} {filename} | python -m json.tool

def grep_sed_json(keyword, lineno, filename):
    !grep {keyword} {filename} | sed -n {lineno}p | python -m json.tool


In [12]:
sql("show tables")

namespace,tableName,isTemporary
default,delta_v1,false
default,delta_v2,false
default,family,false
default,users,false



### Q1. 예제 테이블 없는 경우 데이터 리텐션 강제 실행방법?
* 테이블 생성시 부터 retentionDurationCheck false, deletedFileRetentionDuration 1분 설정
* 임의의 데이터 저장 후, 일부 데이터를 즉시 삭제
* 약 1분 대기 후에, retain 0 hours 설정으로 vacuum 실행
* 1분 이전의 삭제됨을 확인

```sql
CREATE TABLE delta_v1 (id INT, data STRING) 
USING delta 
TBLPROPERTIES (
  'delta.deletedFileRetentionDuration' = '1 minutes',
  'spark.databricks.delta.retentionDurationCheck.enabled' = 'false'
);

INSERT INTO delta_v1 VALUES (1, 'first'), (2, 'second');
DELETE FROM delta_v1 WHERE id = 1;

-- wait 1 minute
VACUUM delta_v1;
```

In [15]:
from pyspark.sql import Row
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType

def dropAndRemoveTable(dbName, tableName):
    location="/home/jovyan/work/spark-warehouse/{}".format(tableName)
    !rm -rf {location}
    sql("DROP TABLE IF EXISTS {}.{}".format(dbName, tableName))

In [16]:
dbName="default"
tableName="delta_v1"
dropAndRemoveTable(dbName, tableName)

sql("""
CREATE TABLE {} (id INT, data STRING) 
USING delta 
TBLPROPERTIES (
  'delta.deletedFileRetentionDuration' = '1 minutes',
  'spark.databricks.delta.retentionDurationCheck.enabled' = 'false'
)""".format(tableName))
# spark.databricks.delta.retentionDurationCheck.enabled

sql("INSERT INTO {} VALUES (1, 'first'), (2, 'second')".format(tableName))
sql("DELETE FROM {} WHERE id = 1".format(tableName))

""


In [17]:
ls("./spark-warehouse/delta_v1/")

total 8
drwxrwxrwx 1 jovyan 1000 512 Oct 29 04:56 .
drwxrwxrwx 1 jovyan 1000 512 Oct 29 04:55 ..
drwxrwxrwx 1 jovyan 1000 512 Oct 29 04:56 _delta_log
-rwxrwxrwx 1 jovyan 1000 382 Oct 29 04:56 part-00000-0e13db32-fe48-4793-826e-df135f4e0762-c000.snappy.parquet
-rwxrwxrwx 1 jovyan 1000  12 Oct 29 04:56 .part-00000-0e13db32-fe48-4793-826e-df135f4e0762-c000.snappy.parquet.crc
-rwxrwxrwx 1 jovyan 1000 694 Oct 29 04:56 part-00000-4a98b7b9-9d2f-470d-a138-8d1ca4d5a8dc-c000.snappy.parquet
-rwxrwxrwx 1 jovyan 1000  16 Oct 29 04:56 .part-00000-4a98b7b9-9d2f-470d-a138-8d1ca4d5a8dc-c000.snappy.parquet.crc
-rwxrwxrwx 1 jovyan 1000 701 Oct 29 04:56 part-00001-e66155f1-5323-4090-ab29-c3c571193ed3-c000.snappy.parquet
-rwxrwxrwx 1 jovyan 1000  16 Oct 29 04:56 .part-00001-e66155f1-5323-4090-ab29-c3c571193ed3-c000.snappy.parquet.crc


In [18]:
# -- wait 1 minute
import time
time.sleep(60)

In [19]:
sql("VACUUM {}".format(tableName))

path
file:/home/jovyan/work/spark-warehouse/delta_v1


In [20]:
ls("./spark-warehouse/{}/".format(tableName))

total 4
drwxrwxrwx 1 jovyan 1000 512 Oct 29  2024 .
drwxrwxrwx 1 jovyan 1000 512 Oct 29 04:55 ..
drwxrwxrwx 1 jovyan 1000 512 Oct 29 04:56 _delta_log
-rwxrwxrwx 1 jovyan 1000 382 Oct 29 04:56 part-00000-0e13db32-fe48-4793-826e-df135f4e0762-c000.snappy.parquet
-rwxrwxrwx 1 jovyan 1000  12 Oct 29 04:56 .part-00000-0e13db32-fe48-4793-826e-df135f4e0762-c000.snappy.parquet.crc
-rwxrwxrwx 1 jovyan 1000 701 Oct 29 04:56 part-00001-e66155f1-5323-4090-ab29-c3c571193ed3-c000.snappy.parquet
-rwxrwxrwx 1 jovyan 1000  16 Oct 29 04:56 .part-00001-e66155f1-5323-4090-ab29-c3c571193ed3-c000.snappy.parquet.crc


In [22]:
sql("select * from {}".format(tableName))

id,data
2,second


### Q2. 예제 테이블 없는 경우 로그 리텐션 강제 실행방법?
* 테이블 생성시 부터 retentionDurationCheck false 설정, logRetentionDuration 1분 설정
* 임의의 데이터를 저장하되 10회 미만의 커밋이 발생하도록 한다 (예 5회, 총 6개의 커밋 발생)
* 약 1분 대기 후에, 명시적으로 checkpoint 메서드를 실행
* 1분 이전에 수행된 로그가 삭제 되었는지 확인

```sql
CREATE TABLE delta_v2 (id INT, data STRING) 
USING delta 
TBLPROPERTIES (
  'delta.logRetentionDuration' = '1 minutes',
  'spark.databricks.delta.retentionDurationCheck.enabled' = 'false'
);

INSERT INTO delta_v2 VALUES (1, 'first'), (2, 'second');

-- wait 1 minute
OPTIMIZE delta_v2;
CHECKPOINT delta_v2;
```

* 임의의 테이블을 log retention 지정하지 않고 생성하는 경우 30일 log-retention 이므로 이 때에 11건을 저장한다
* 그리고 log-retention 1일로 변경하고, 다시 11건을 집어 넣고 익일 오전 까지 대기
* 익일 오전 8시에 다시 11건을 입력하고 모니터링 했을 때에 로그 상태를 보고
* 다시 오전 9시에 11건을 입력하고 모니터링 했을 때에 로그 상태를 보면 모든 테스트가 완료된다

In [18]:
dbName="default"
tableName="delta_v2"
dropAndRemoveTable(dbName, tableName)

sql("""
CREATE TABLE {}.{} (id INT, data STRING) 
USING delta 
""".format(dbName, tableName))

for id in range(0, 11):
    sql("INSERT INTO {}.{} VALUES ({}, 'data-{}')".format(dbName, tableName, id, id))
sql("select count(1) from {}.{}".format(dbName, tableName))

count(1)
11


In [19]:
ls("./spark-warehouse/{}/_delta_log".format(tableName))

total 64
drwxrwxrwx 1 jovyan 1000   512 Oct 29 09:20 .
drwxrwxrwx 1 jovyan 1000   512 Oct 29 09:20 ..
-rwxrwxrwx 1 jovyan 1000   771 Oct 29 09:20 00000000000000000000.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000000.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000001.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000001.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000002.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000002.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000003.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000003.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000004.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000004.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000005.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000005.json.crc
-rwxrwxrwx 1 jovyan 1000   6

In [21]:
show("describe history {}.{}".format(dbName, tableName))
show("show tblproperties {}.{}".format(dbName, tableName))

+-------+-----------------------+------+--------+------------+-----------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                          |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                          |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+------------+-----------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------+------------+-----------------------------------+
|11     |2024-10-29 18:20:19.877|null  |null    |WRITE    

In [22]:
spark.conf.set("delta.logRetentionDuration", "1 days")
sql("""
ALTER TABLE {}.{}
SET TBLPROPERTIES (
  'delta.logRetentionDuration' = '1 minutes'
)
""".format(dbName, tableName))

for id in range(11, 22):
    sql("INSERT INTO {}.{} VALUES ({}, 'data-{}')".format(dbName, tableName, id, id))

sql("select count(1) from {}.{}".format(dbName, tableName))

count(1)
22


In [23]:
# 2024/10/30 오전 9시 이전에 아래의 명령어 수행

for id in range(22, 33):
    sql("INSERT INTO {}.{} VALUES ({}, 'data-{}')".format(dbName, tableName, id, id))

sql("select count(1) from {}.{}".format(dbName, tableName))

# 아래의 경로에 log 가 삭제되지 않았음을 확인
ls("./spark-warehouse/{}/_delta_log".format(tableName))

total 180
drwxrwxrwx 1 jovyan 1000   512 Oct 29  2024 .
drwxrwxrwx 1 jovyan 1000   512 Oct 29  2024 ..
-rwxrwxrwx 1 jovyan 1000   771 Oct 29 09:20 00000000000000000000.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000000.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000001.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000001.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000002.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000002.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000003.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000003.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000004.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000004.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000005.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000005.json.crc
-rwxrwxrwx 1 jovyan 1000   

In [25]:
!date

Wed 30 Oct 2024 12:04:10 AM UTC


In [35]:
# 2024/10/30 오전 9시 이후에 아래의 명령어 수행

for id in range(33, 44):
    sql("INSERT INTO {}.{} VALUES ({}, 'data-{}')".format(dbName, tableName, id, id))

sql("select count(1) from {}.{}".format(dbName, tableName))

# 처음 생성한 로그는 삭제되면 안 될 듯 ...
ls_and_head("./spark-warehouse/{}/_delta_log".format(tableName), 10)

total 240
drwxrwxrwx 1 jovyan 1000   512 Oct 30 00:08 .
drwxrwxrwx 1 jovyan 1000   512 Oct 30 00:08 ..
-rwxrwxrwx 1 jovyan 1000   771 Oct 29 09:20 00000000000000000000.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000000.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000001.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000001.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000002.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 09:20 .00000000000000000002.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000003.json


In [52]:
ls_and_head("./spark-warehouse/{}/_delta_log".format(tableName), 50)

total 244
drwxrwxrwx 1 jovyan 1000   512 Oct 30 00:21 .
drwxrwxrwx 1 jovyan 1000   512 Oct 30 00:08 ..
-rwxrwxrwx 1 jovyan 1000   771 Oct 29 09:20 00000000000000000000.json
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000001.json
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000002.json
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000003.json
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000004.json
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000005.json
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000006.json
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000007.json
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000008.json
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000009.json
-rwxrwxrwx 1 jovyan 1000 11639 Oct 29 09:20 00000000000000000010.checkpoint.parquet
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 09:20 00000000000000000010.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 29 09:20

In [44]:
df = spark.read.parquet("./spark-warehouse/{}/_delta_log/00000000000000000010.checkpoint.parquet".format(tableName))
df.printSchema()
df.orderBy(asc("txn.version")).show(100, truncate=False)

root
 |-- txn: struct (nullable = true)
 |    |-- appId: string (nullable = true)
 |    |-- version: long (nullable = true)
 |    |-- lastUpdated: long (nullable = true)
 |-- add: struct (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- partitionValues: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- modificationTime: long (nullable = true)
 |    |-- dataChange: boolean (nullable = true)
 |    |-- tags: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |    |-- stats: string (nullable = true)
 |-- remove: struct (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- deletionTimestamp: long (nullable = true)
 |    |-- dataChange: boolean (nullable = true)
 |    |-- extendedFileMetadata: boolean (nullable = true)
 |    |-- partitionValues: map (nullable = true)
 |    |    |-- key: string
 |    | 

In [51]:
# show("describe history {}.{}".format(dbName, tableName))
show("show tblproperties {}.{}".format(dbName, tableName))

+--------------------------+---------+
|key                       |value    |
+--------------------------+---------+
|Type                      |MANAGED  |
|delta.logRetentionDuration|1 minutes|
|delta.minReaderVersion    |1        |
|delta.minWriterVersion    |2        |
+--------------------------+---------+



In [53]:
# 2024/10/30 오전 9시 retry

for id in range(44, 55):
    sql("INSERT INTO {}.{} VALUES ({}, 'data-{}')".format(dbName, tableName, id, id))

sql("select count(1) from {}.{}".format(dbName, tableName))

ls_and_head("./spark-warehouse/{}/_delta_log".format(tableName), 60)

total 128
drwxrwxrwx 1 jovyan 1000   512 Oct 30  2024 .
drwxrwxrwx 1 jovyan 1000   512 Oct 30  2024 ..
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000034.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000035.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000036.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000037.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000038.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000039.json
-rwxrwxrwx 1 jovyan 1000 13518 Oct 30 00:08 00000000000000000040.checkpoint.parquet
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000040.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000041.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000042.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000043.json
-rwxrwxrwx 1 jovyan 1000   698 Oct 30 00:08 00000000000000000044.json
-rwxrwxrwx 1 jovyan 1000   763 Oct 30 00:21

In [13]:
# -- wait 1 minute
import time
time.sleep(60)

for id in range(11, 22):
    sql("INSERT INTO {}.{} VALUES ({}, 'data-{}')".format(dbName, tableName, id, id))
sql("select count(1) from {}.{}".format(dbName, tableName))

count(1)
22


In [15]:
sql("OPTIMIZE {}.{}".format(dbName, tableName))
sql("VACUUM {}.{} RETAIN 0 HOURS".format(dbName, tableName))
ls("./spark-warehouse/{}/_delta_log".format(tableName))

total 132
drwxrwxrwx 1 jovyan 1000   512 Oct 29 06:31 .
drwxrwxrwx 1 jovyan 1000   512 Oct 29  2024 ..
-rwxrwxrwx 1 jovyan 1000   855 Oct 29 06:29 00000000000000000000.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 06:29 .00000000000000000000.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 06:29 00000000000000000001.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 06:29 .00000000000000000001.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 06:29 00000000000000000002.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 06:29 .00000000000000000002.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 06:29 00000000000000000003.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 06:29 .00000000000000000003.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 06:29 00000000000000000004.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 06:29 .00000000000000000004.json.crc
-rwxrwxrwx 1 jovyan 1000   693 Oct 29 06:29 00000000000000000005.json
-rwxrwxrwx 1 jovyan 1000    16 Oct 29 06:29 .00000000000000000005.json.crc
-rwxrwxrwx 1 jovyan 1000   

### Q3. 특정 값을 가진 레코드 전체 및 히스토리 정보까지 완전히 삭제하려면?
* 테이블 생성 시에 스파크 세션 수준에서 'retentionDurationCheck.enabled' false 설정
* 대상 테이블에서 임의의 로우를 가진 데이터를 삭제
* `retain 0 hours` 옵션으로 `vacuum` 수행
* 과거 데이터 및 히스토리 정보 확인

```sql
val spark = SparkSession.builder()
  .appName("GDPR Retention Session")
  .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")
  .getOrCreate()

spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

spark.sql("DELETE FROM delta_v3 WHERE phone IS NOT NULL")
spark.sql("VACUUM delta_v3 RETAIN 0 HOURS")
```

In [30]:
df_v2.write.format("delta").mode("append").option("mergeSchema", True).saveAsTable("{}.{}".format(dbName, tableName))

sql("show tables")
show("describe extended {}.{}".format(dbName, tableName), 100)

+----------------------------+----------------------------------------------------------------+-------+
|col_name                    |data_type                                                       |comment|
+----------------------------+----------------------------------------------------------------+-------+
|id                          |int                                                             |       |
|firstName                   |string                                                          |       |
|lastName                    |string                                                          |       |
|middleName                  |string                                                          |       |
|                            |                                                                |       |
|# Partitioning              |                                                                |       |
|Not partitioned             |                                  

### Q4. 특정 값을 가진 컬럼을 제거하고 히스토리 정보까지 완전히 삭제하려면?
* 대상 테이블에서 임의의 로우를 가진 컬럼을 제거한 데이터 프레임 생성
* 기존 경로에 `overwrite` 모드와 `overwriteSchema` 옵션으로 데이터를 덮어쓴다
* 기존 데이터 및 히스토리 정보 확인

```sql
val originalData = spark.read.format("delta").load("/path/to/delta_v4")
val delta_v4 = originalData.drop("phone")
delta_v4.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/path/to/delta_v4")
```

In [33]:
sql("select * from {}.{}".format(dbName, tableName))

id,firstName,lastName
1,suhyuk,park
2,youngmi,kim


In [35]:
# Q1 예제를 다시 실행하고

schema_v3 = StructType([
    StructField("id", IntegerType(), True),
    StructField("firstName", StringType(), True),
    StructField("middleName", StringType(), True),
    StructField("lastName", StringType(), True)
])
rows_v3 = []
rows_v3.append(Row(3, "sowon", "eva", "park"))
rows_v3.append(Row(4, "sihun", "sean", "park"))
df_v3 = spark.createDataFrame(rows_v3, schema_v3)
df_v3.write.format("delta").mode("overwrite").option("overwriteSchema", True).saveAsTable("{}.{}".format(dbName, tableName))

sql("show tables")
show("describe extended {}.{}".format(dbName, tableName), 100)

+----------------------------+----------------------------------------------------------------+-------+
|col_name                    |data_type                                                       |comment|
+----------------------------+----------------------------------------------------------------+-------+
|id                          |int                                                             |       |
|firstName                   |string                                                          |       |
|middleName                  |string                                                          |       |
|lastName                    |string                                                          |       |
|                            |                                                                |       |
|# Partitioning              |                                                                |       |
|Not partitioned             |                                  

In [36]:
sql("select * from {}.{}".format(dbName, tableName))

id,firstName,middleName,lastName
4,sihun,sean,park
3,sowon,eva,park


### Q5. `VACUUM` 실행 시에도 `RETAIN 100 HOURS` 와 같이 적용할 수 있는데 원 테이블 설정과 런타임 시의 설정 중에 어떤 것 기준으로 적용이 되는가?
* 테이블 설정보다 높은 값이 아닌 경우 오류를 반환하게 됩니다
* `spark.databricks.delta.retentionDurationCheck.enabled = false` 설정 시에는 지정이 가능하며 반드시 `RETAIN` 설정 값을 지정해야 합니다
* 조회되는 가장 긴 기간 혹은 업데이트 가능성이 있는 시간 보다 긴 시간을 지정해야 데이터 유실을 막을 수 있습니다


In [37]:
dropAndRemoveTable(dbName, tableName)

schema_v4 = StructType([
    StructField("id", IntegerType(), True),
    StructField("firstName", StringType(), True),
    StructField("lastName", StringType(), True)
])
rows_v4 = []
rows_v4.append(Row(1, "suhyuk", "park"))
rows_v4.append(Row(3, "sowon", "park"))
rows_v4.append(Row(4, "sean", "park"))

df_v4 = spark.createDataFrame(rows_v1, schema_v1)
df_v4.write.format("delta").mode("overwrite").partitionBy("lastName").saveAsTable("{}.{}".format(dbName, tableName))

show("describe extended {}.{}".format(dbName, tableName), 100)
sql("select * from {}.{}".format(dbName, tableName))

+----------------------------+----------------------------------------------------------------+-------+
|col_name                    |data_type                                                       |comment|
+----------------------------+----------------------------------------------------------------+-------+
|id                          |int                                                             |       |
|firstName                   |string                                                          |       |
|lastName                    |string                                                          |       |
|                            |                                                                |       |
|# Partitioning              |                                                                |       |
|Part 0                      |lastName                                                        |       |
|                            |                                  

id,firstName,lastName
2,youngmi,kim
1,suhyuk,park


In [39]:
schema_v4a = StructType([
    StructField("id", IntegerType(), True),
    StructField("firstName", StringType(), True),
    StructField("middleName", StringType(), True),
    StructField("lastName", StringType(), True)
])
rows_v4a = []
rows_v4a.append(Row(2, "youngmi", "kiki", "kim"))
df_v4a = spark.createDataFrame(rows_v4a, schema_v4a)
df_v4a.write.format("delta").mode("append").option("mergeSchema", True).partitionBy("lastName").saveAsTable("{}.{}".format(dbName, tableName))

In [42]:
show("describe extended {}.{}".format(dbName, tableName), 100)

+----------------------------+----------------------------------------------------------------+-------+
|col_name                    |data_type                                                       |comment|
+----------------------------+----------------------------------------------------------------+-------+
|id                          |int                                                             |       |
|firstName                   |string                                                          |       |
|lastName                    |string                                                          |       |
|middleName                  |string                                                          |       |
|                            |                                                                |       |
|# Partitioning              |                                                                |       |
|Part 0                      |lastName                          